In [ ]:
import cv2
import os
import subprocess
import numpy as np
import glob
import time

### Calibrating the camera using pre-taken images of chess board

In [ ]:
import numpy as np
import cv2
import glob
ts= time.time()
# Termination criteria for corner sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points based on the grid size (squares of 1 unit size)
grid_size = (7, 7)  # For an 8x8 chessboard, the inner corners grid is 7x7
objp = np.zeros((grid_size[0] * grid_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:grid_size[0], 0:grid_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all images
objpoints = []
imgpoints = []

# Load images from the specified folder
image_paths = glob.glob('../../Figures/Cam_2/*.jpg')  # Update with your image paths

for fname in image_paths:
    img = cv2.imread(fname)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, grid_size, None)
    
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        
        # Draw and display the corners (optional)
        cv2.drawChessboardCorners(img, grid_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(100)  # Adjust the delay as needed
    
    else:
        print(f"Chessboard corners not found in image {fname}")

cv2.destroyAllWindows()

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
te = time.time()
t=te-ts
print(t)
print("Camera matrix:\n", mtx)
print("Distortion coefficients:\n", dist)


**Obtained matrices and coefficients for cam 2**

Camera matrix:  \
 [[500.23727865,   0.       ,         315.2966858 ] \
 [  0.,            499.65909552, 251.36140266]  \
 [  0.,           0.,           1.        ]]  \
Distortion coefficients:  \
 [[-0.48160835,  0.34932479,  0.00271425, -0.00349605, -0.20166287]] 

In [ ]:
import cv2
import numpy as np

# Camera matrix and distortion coefficients from calibration
mtx = np.array([[500.23727865, 0., 315.2966858],
                [0., 499.65909552, 251.36140266],
                [0., 0., 1.]])
dist = np.array([[-0.48160835, 0.34932479, 0.00271425, -0.00349605, -0.20166287]])

# Load the image to be undistorted
img = cv2.imread('captured_image.jpg')  # Replace with your image path

# Undistort the image
h, w = img.shape[:2]
newcameramtx, _ = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Use cv2.initUndistortRectifyMap and cv2.remap to undistort without cropping
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
undistorted_img = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# Display the original and undistorted images side by side
cv2.imshow('Original Image', img)
cv2.imshow('Undistorted Image', undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the undistorted image
cv2.imwrite('undistorted_image.jpg', undistorted_img)
